# 03 - Analyse IA avec Google Gemini

Nous entrons dans le cœur du sujet : l'intelligence artificielle.
Nous allons utiliser **Google Gemini** pour analyser nos avis.

### Objectifs :
1. Comprendre comment appeler l'API Gemini.
2. Gérer les limites de tokens (contexte).
3. Passer d'une analyse texte libre à une donnée structurée (JSON) exploitable.

## 1. Configuration de l'API

Pour utiliser le modèle, il faut une clé API.
Vous pouvez la récupérer gratuitement sur [Google AI Studio](https://aistudio.google.com/).

Une fois la clé obtenue, le mieux est de la stocker dans un fichier `.env` ou directement dans les secrets de votre environnement.

In [ ]:
import google.generativeai as genai
import os
import pandas as pd
from dotenv import load_dotenv

# Chargement des variables d'environnement
load_dotenv()

GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')

# Configuration globale de la librairie
genai.configure(api_key=GOOGLE_API_KEY)

## 2. Chargement des Données

Reprenons nos avis consolidés.

In [ ]:
sample_reviews

In [ ]:
df = pd.read_csv('df_final.csv')

# On garde un échantillon représentatif pour la démo, ou tout le dataset si c'est petit
# Pour l'IA, on va concaténer le texte des avis pour lui donner du contexte

# Prenons les 10 derniers avis pour tester
sample_reviews = df.head(10)

# Création d'un gros bloc de texte avec tous les avis
reviews_text = ""
for index, row in sample_reviews.iterrows():
    reviews_text += f"[Note: {row['rating']}/5] {row['text']}\n---\n"

print(f"Texte préparé ({len(reviews_text)} caractères) :")
print(reviews_text[:500] + "... [tronqué]")

## 3. Premier Appel Simple (Modèle Flash)

Nous allons utiliser le modèle `gemini-2.5-flash`. C'est le modèle le plus rapide et le moins cher, idéal pour du gros volume.

Faisons un test naïf : "Analyse ça".

In [ ]:
model_flash = genai.GenerativeModel('gemini-2.5-flash')

prompt_basic = f"Analyse ces avis :\n{reviews_text}"

response = model_flash.generate_content(prompt_basic)

print(response.text)

## 4. La Limite de Tokens

Chaque modèle a une "fenêtre de contexte". C'est la quantité de texte qu'il peut lire en une fois.
- **Gemini 2.5 Flash** : 1 Million de tokens (~700 000 mots)
- **Gemini 2.5 Pro** : 1 Million de tokens

C'est énorme ! Cela signifie qu'on peut littéralement lui donner des livres entiers.
Mais attention, plus on envoie de texte, plus c'est lent et coûteux (si version payante).

### Vérifions la taille de notre prompt

In [ ]:
# L'API permet de compter les tokens avant d'envoyer
token_count = model_flash.count_tokens(prompt_basic)
print(f"Nombre de tokens : {token_count.total_tokens}")
print(f"Limite du modèle : 1 000 000")

if token_count.total_tokens > 1000000:
    print("⚠️ Attention, trop de texte !")
else:
    print("✅ Ça passe large !")

## 5. Structurer la Sortie (JSON)

Le résultat du point 3 était du texte libre. C'est sympa à lire, mais impossible à mettre dans un graphique.
On va demander au modèle Flash de nous répondre en **JSON**.

In [ ]:
prompt_json = f"""
Analyse ces avis et sors-moi les 3 points forts et les 3 points faibles.
Réponds uniquement au format JSON comme ceci :
{{
  "points_forts": ["pf1", "pf2", "pf3"],
  "points_faibles": ["pf1", "pf2", "pf3"]
}}

AVIS :
{reviews_text}
"""

response_json = model_flash.generate_content(prompt_json)

response_json.text

## 6. La Méthode Robuste : Pydantic & Gemini 2.5 Pro

Pour une application en production, on ne peut pas juste "espérer" que le modèle respecte le JSON.
On utilise le mode **Structured Output** avec le modèle `gemini-2.5-pro` (plus intelligent).
On définit la structure exacte voulue avec `Pydantic`.

In [ ]:
from pydantic import BaseModel, Field
from typing import List

# 1. On définit l'architecture de la réponse
class Theme(BaseModel):
    nom: str = Field(description="Nom court de la thématique (ex: Bug, UX, Prix)")
    sentiment: str = Field(description="Positif ou Négatif")
    citation: str = Field(description="Une citation courte tirée des avis qui illustre le thème")

class AnalyseResponse(BaseModel):
    synthese: str = Field(description="Résumé général en 2 phrases")
    themes: List[Theme] = Field(description="Liste des thèmes principaux identifiés")

# 2. On instancie le modèle PRO
model_pro = genai.GenerativeModel('gemini-2.5-pro')

# 3. Appel avec le schéma forcé
prompt_struct = f"Analyse ces avis en profondeur :\n{reviews_text}"

response_struct = model_pro.generate_content(
    prompt_struct,
    generation_config={
        "response_mime_type": "application/json",
        "response_schema": AnalyseResponse
    }
)

print(response_struct.text)

## 7. Exploitation

On peut maintenant charger ce JSON directement en Python.

In [ ]:
import json

# Parsing du JSON
data = json.loads(response_struct.text)

# Affichage propre
print(f"📢 SYNTHÈSE : {data['synthese']}\n")

for theme in data['themes']:
    icon = "✅" if theme['sentiment'] == "Positif" else "❌"
    print(f"{icon} {theme['nom']} : \"{theme['citation']}\"")